In [1]:
import sys
sys.path.insert(0, "../..")

import torch
import torch.nn as nn
from src.data import make_dataset
from pathlib import Path
from loguru import logger

In [2]:
datadir = Path("../../data/raw/")
batch_size = 64
train_dataloader, test_dataloader = make_dataset.get_MNIST(datadir, batch_size=batch_size) 

In [3]:
datadir.resolve()

PosixPath('/Users/raoulgrouls/code/ML22/data/raw')

In [4]:
len(train_dataloader), len(test_dataloader)

(938, 157)

We can obtain an item:

In [5]:
x, y = next(iter(train_dataloader))
x.shape, y.shape

(torch.Size([64, 1, 28, 28]), torch.Size([64]))

The image follows the channels-first convention: (channel, width, height). The label is an integer.

In [6]:
import torch
from torch import nn

# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class CNN(nn.Module):
    def __init__(self, filters, units1, units2, input_size=(32, 1, 28, 28)):
        super().__init__()

        self.convolutions = nn.Sequential(
            nn.Conv2d(1, filters, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filters, filters, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filters, filters, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
        )

        activation_map_size = self._conv_test(input_size)
        logger.info(f"Aggregating activationmap with size {activation_map_size}")
        self.agg = nn.AvgPool2d(activation_map_size)

        self.dense = nn.Sequential(
            nn.Flatten(),
            nn.Linear(filters, units1),
            nn.ReLU(),
            nn.Linear(units1, units2),
            nn.ReLU(),
            nn.Linear(units2, 10)
        )

    def _conv_test(self, input_size = (32, 1, 28, 28)):
        x = torch.ones(input_size)
        x = self.convolutions(x)
        return x.shape[-2:]

    def forward(self, x):
        x = self.convolutions(x)
        x = self.agg(x)
        logits = self.dense(x)
        return logits

model = CNN(filters=32, units1=128, units2=64).to(device)

2023-02-10 13:34:59.872 | INFO     | __main__:__init__:26 - Aggregating activationmap with size torch.Size([2, 2])


Using cpu device


In [7]:
from torchsummary import summary
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
              ReLU-2           [-1, 32, 28, 28]               0
         MaxPool2d-3           [-1, 32, 14, 14]               0
            Conv2d-4           [-1, 32, 12, 12]           9,248
              ReLU-5           [-1, 32, 12, 12]               0
         MaxPool2d-6             [-1, 32, 6, 6]               0
            Conv2d-7             [-1, 32, 4, 4]           9,248
              ReLU-8             [-1, 32, 4, 4]               0
         MaxPool2d-9             [-1, 32, 2, 2]               0
        AvgPool2d-10             [-1, 32, 1, 1]               0
          Flatten-11                   [-1, 32]               0
           Linear-12                  [-1, 128]           4,224
             ReLU-13                  [-1, 128]               0
           Linear-14                   

In [8]:
import torch.optim as optim
from src.models import metrics
optimizer = optim.Adam
loss_fn = torch.nn.CrossEntropyLoss()
accuracy = metrics.Accuracy()

In [9]:
yhat = model(x)
accuracy(y, yhat)

tensor(0.1250)

We now have everything we need to train the model.

In [10]:
import mlflow
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("mnist_convolutions")

<Experiment: artifact_location='./mlruns/1', creation_time=1675954831805, experiment_id='1', last_update_time=1675954831805, lifecycle_stage='active', name='mnist_convolutions', tags={}>

In [11]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [12]:
import torch.optim as optim
from src.models import metrics
from src.models import train_model
from datetime import datetime
modeldir = Path("./models")

def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "convnet")
        mlflow.set_tag("dev", "raoul")
        mlflow.log_params(params)
        mlflow.log_param("datadir", f"{datadir.resolve()}")
        mlflow.log_param("batchsize", f"{batch_size}")

        optimizer = optim.Adam
        loss_fn = torch.nn.CrossEntropyLoss()
        accuracy = metrics.Accuracy()
        model = CNN(**params)
        model = train_model.trainloop(
            epochs=3,
            model=model,
            optimizer=optimizer,
            learning_rate=1e-3,
            loss_fn=loss_fn,
            metrics=[accuracy],
            train_dataloader=train_dataloader,
            test_dataloader=test_dataloader,
            log_dir="modellog",
            train_steps=100, #len(train_dataloader),
            eval_steps=100, #len(test_dataloader),
        )

        tag = datetime.now().strftime("%Y%m%d-%H%M")
        modelpath = modeldir / (tag + "model.pt")
        torch.save(model, modelpath)
        mlflow.log_artifact(local_path=modelpath, artifact_path="pytorch_models")
        return {'loss' : test_loss, 'status': STATUS_OK}

In [13]:
search_space = {
    'filters' : scope.int(hp.quniform('filters', 16, 128, 8)),
    'units1' : scope.int(hp.quniform('units1', 16, 128, 8)),
    'units2' : scope.int(hp.quniform('units2', 16, 128, 8)),
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=10,
    trials=Trials()
)

  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

2023-02-10 13:35:26.522 | INFO     | __main__:__init__:26 - Aggregating activationmap with size torch.Size([2, 2])
2023-02-10 13:35:26.524 | INFO     | src.data.data_tools:dir_add_timestamp:129 - Logging to modellog/20230210-1335
100%|##########| 100/100 [00:04<00:00, 20.77it/s]
Session not detected. You should not be calling `report` outside `tuner.fit()` or while using the class API. 

  File "/Users/raoulgrouls/.pyenv/versions/3.10.9/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Users/raoulgrouls/.pyenv/versions/3.10.9/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/Users/raoulgrouls/Library/Caches/pypoetry/virtualenvs/deep-learning-wM7qE7ca-py3.10/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/Users/raoulgrouls/Library/Caches/pypoetry/virtualenvs/deep-learning-wM7qE7ca-py3.10/lib/python3.10/site-packages/traitlet

 10%|█         | 1/10 [00:19<02:55, 19.54s/trial, best loss: 0.8182338976860046]

2023-02-10 13:35:46.011 | INFO     | __main__:__init__:26 - Aggregating activationmap with size torch.Size([2, 2])
2023-02-10 13:35:46.012 | INFO     | src.data.data_tools:dir_add_timestamp:129 - Logging to modellog/20230210-1335
100%|##########| 100/100 [00:06<00:00, 15.72it/s]
2023-02-10 13:35:54.299 | INFO     | src.models.train_model:trainloop:180 - Epoch 0 train 1.4701 test 0.9619 metric ['0.6130']
100%|##########| 100/100 [00:06<00:00, 15.80it/s]
2023-02-10 13:36:02.562 | INFO     | src.models.train_model:trainloop:180 - Epoch 1 train 0.8506 test 0.7854 metric ['0.7055']
100%|##########| 100/100 [00:06<00:00, 15.83it/s]
2023-02-10 13:36:10.848 | INFO     | src.models.train_model:trainloop:180 - Epoch 2 train 0.7720 test 0.7254 metric ['0.7286']
100%|##########| 3/3 [00:24<00:00,  8.28s/it]


 20%|██        | 2/10 [00:44<03:01, 22.68s/trial, best loss: 0.7253523311018943]

...

 80%|████████  | 8/10 [02:11<00:31, 15.59s/trial, best loss: 0.7253523311018943]

2023-02-10 13:37:37.537 | INFO     | __main__:__init__:26 - Aggregating activationmap with size torch.Size([2, 2])
2023-02-10 13:37:37.538 | INFO     | src.data.data_tools:dir_add_timestamp:129 - Logging to modellog/20230210-1337
100%|##########| 100/100 [00:03<00:00, 29.71it/s]
2023-02-10 13:37:41.904 | INFO     | src.models.train_model:trainloop:180 - Epoch 0 train 1.7543 test 1.0782 metric ['0.5866']
100%|##########| 100/100 [00:03<00:00, 29.96it/s]
2023-02-10 13:37:46.239 | INFO     | src.models.train_model:trainloop:180 - Epoch 1 train 0.9987 test 1.0304 metric ['0.6227']
100%|##########| 100/100 [00:03<00:00, 30.33it/s]
2023-02-10 13:37:50.535 | INFO     | src.models.train_model:trainloop:180 - Epoch 2 train 0.8930 test 0.8504 metric ['0.6880']
100%|##########| 3/3 [00:12<00:00,  4.33s/it]


 90%|█████████ | 9/10 [02:24<00:14, 14.79s/trial, best loss: 0.7253523311018943]

2023-02-10 13:37:50.574 | INFO     | __main__:__init__:26 - Aggregating activationmap with size torch.Size([2, 2])
2023-02-10 13:37:50.575 | INFO     | src.data.data_tools:dir_add_timestamp:129 - Logging to modellog/20230210-1337
100%|##########| 100/100 [00:07<00:00, 13.85it/s]
2023-02-10 13:38:00.170 | INFO     | src.models.train_model:trainloop:180 - Epoch 0 train 1.6905 test 1.0599 metric ['0.5980']
100%|##########| 100/100 [00:07<00:00, 13.85it/s]
2023-02-10 13:38:09.743 | INFO     | src.models.train_model:trainloop:180 - Epoch 1 train 0.8973 test 0.8911 metric ['0.6420']
100%|##########| 100/100 [00:07<00:00, 13.81it/s]
2023-02-10 13:38:19.428 | INFO     | src.models.train_model:trainloop:180 - Epoch 2 train 0.7836 test 0.7676 metric ['0.7128']
100%|##########| 3/3 [00:28<00:00,  9.62s/it]


100%|██████████| 10/10 [02:53<00:00, 17.30s/trial, best loss: 0.7253523311018943]


In [14]:
best_result

{'filters': 88.0, 'units1': 96.0, 'units2': 48.0}